In [ ]:
import pixellib
from pixellib.tune_bg import alter_bg
import cv2
import numpy as np

# Name of the image file
img_name = "vin.jpg"

# Converts Stock Image to grey background Image
change_bg = alter_bg(model_type = "pb")
change_bg.load_pascalvoc_model("xception_pascalvoc.pb")
change_bg.gray_bg(img_name,output_image_name="gray_img.jpg", detect = "person")
change_bg.blur_bg("gray_img.jpg", low = True, detect = "person", output_image_name="blur_img.jpg")

# Read images of before and after
img1 = cv2.imread(img_name)
img2 = cv2.imread(r'blur_img.jpg')

# Resize Images
percent = 60
width = int(img1.shape[1]* percent / 100)
height = int(img1.shape[0]*percent / 100)
dim = (width, height)

img1r = cv2.resize(img1, dim)
img2r = cv2.resize(img2, dim)

# Custom filter to sharp, Increase Exposure by 1.2
filter1 = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])   # Increase Sharpness
filter2 = np.array([[0, 0, 0], [0, 1.2, 0], [0, 0, 0]])     # Increase Exposure by 1.2

ImageMod1 = cv2.blur(cv2.filter2D(img2r,-1,filter1), (2,2))          # Slightly blurs+Sharpens
custom_image = cv2.filter2D(ImageMod1,-1,filter2)                    # Exposure
img2r_enh = cv2.detailEnhance(custom_image, sigma_s=1, sigma_r=0.07) # Adds Slight details

# Adaptive Histogram
lab = cv2.cvtColor(img2r_enh, cv2.COLOR_BGR2LAB)
lab_planes = cv2.split(lab)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(10,10))
lab_planes[0] = clahe.apply(lab_planes[0])
lab = cv2.merge(lab_planes)
img2r_hst = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

# bileteral filter to smooten out
img2r_blt = cv2.bilateralFilter(img2r_hst,15,50,50)

# Display Both Images on Seperate window
cv2.imshow('Stock & Blurred Enhanced', np.hstack([img2r_hst, img2r_blt]))
cv2.waitKey(0)
cv2.destroyAllWindows() 
